In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import re

In [3]:
filepath = '/home/anhduc/Downloads/Telegram Desktop/traindatatopic.txt'

In [4]:
f = open(filepath,encoding='utf8')

In [5]:
str = f.read()

# Preprocessing

In [6]:
label = re.findall(r'<label>(.*?)</label>', str)
content = re.findall(r'<content>\s*((?:.|\n)*?)</content>', str)
print(len(label))
print(len(content))

4965
4965


In [7]:
for i in range(len(content)):
    content[i] = re.sub(r'[^\w\s]','',content[i])

In [8]:
from underthesea import word_tokenize

In [9]:
content

['Cá tính mà rất đáng yêu nè\nSee Translation',
 'HOT HOT HOT \nThe Voice tối nay sẽ có sự xuất hiện của ĐINH HƯƠNG với vai trò là MC HẬU TRƯỜNG nhe cả nhàaaaaaaa\nĐón xem nào heheeeee\nSee Translation',
 'Tin HOT nè cả nhà ơiiiiiiiiiiiiiii\nĐINH HƯƠNG nhà mình đã nhận giải CA SỸ CỦA NĂM của TheBox Idol đóooooooooooooooooo\n\nCùng AD gửi lời chúc mừng đến Đinh Hương nào \nSee Translation',
 'Hôn nhân có thực sự đáng sợ thế không\nSee Translation',
 'Ảnh Đinh Hương chụp khi tổng duyệt Thebox Idol hôm nay nè nghen \nHóng ảnh và clip Đinh Hương nhận giải CA SỸ CỦA NĂM\nNghe nói là nàng đẹp lộng lẫy luôn \nHôm nay cô nàng cũng tham gia trình diễn với ca khúc SO I đóa\n\nhóng hóng hóng\nSee Translation',
 'ANH THÍCH VỢ NGU HAY VỢ THÔNG MINH\nHôm rồi tôi có trả lời đài tiếng nói Bình Dương khi bạn phóng viên hỏi đại ý rằng Tại sao đàn ông Việt rất kỵ vợ thông minh Tôi bảo Ai kỵ chứ tôi không kỵ Trái lại tôi thích lấy vợ thông minh Nghĩ đùa một chút rằng Cũng là một kiếp làm chồng như nhau là

In [10]:
for i in range(len(content)):
    content[i] = word_tokenize(content[i], format="text")
content

['Cá_tính mà rất đáng yêu nè See_Translation',
 'HOT HOT_HOT_The_Voice tối nay sẽ có sự xuất_hiện của ĐINH_HƯƠNG với vai_trò là MC HẬU_TRƯỜNG nhe cả nhàaaaaaaa Đón xem nào heheeeee See_Translation',
 'Tin HOT nè cả nhà ơiiiiiiiiiiiiiii ĐINH_HƯƠNG nhà mình đã nhận giải CA SỸ CỦA NĂM của TheBox_Idol đóooooooooooooooooo Cùng AD gửi lời chúc_mừng đến Đinh_Hương nào See_Translation',
 'Hôn_nhân có thực_sự đáng sợ thế không See_Translation',
 'Ảnh Đinh_Hương chụp khi tổng_duyệt Thebox_Idol hôm_nay nè nghen Hóng ảnh và clip Đinh_Hương nhận giải CA SỸ CỦA NĂM Nghe nói là nàng đẹp lộng_lẫy luôn Hôm_nay cô nàng cũng tham_gia trình_diễn với ca_khúc SO I đóa hóng hóng hóng See_Translation',
 'ANH THÍCH VỢ NGU HAY VỢ THÔNG_MINH Hôm rồi tôi có trả_lời đài tiếng_nói Bình_Dương khi bạn phóng_viên hỏi đại_ý rằng Tại_sao đàn_ông Việt rất kỵ vợ thông_minh Tôi bảo Ai_kỵ chứ tôi không kỵ Trái_lại tôi thích lấy vợ thông_minh Nghĩ đùa một_chút rằng Cũng là một kiếp làm chồng như nhau làm chồng một cô vợ thôn

In [11]:
import pandas as pd

In [12]:
df = pd.DataFrame()
df['content'] = content
df['label'] = label

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4965 entries, 0 to 4964
Data columns (total 2 columns):
content    4965 non-null object
label      4965 non-null object
dtypes: object(2)
memory usage: 77.7+ KB


In [14]:
labels = df['label'].value_counts()
labels

sentiment           2552
advertisement       1337
other topics         393
purchase             309
recruit              189
foreign language     184
oreign language        1
Name: label, dtype: int64

# Resampling

In [15]:
from sklearn.utils import resample

In [16]:
df_majority = df[df.label=='sentiment']

In [17]:
df_imbalanced1 = df[df.label=='advertisement']
df_imbalanced2 = df[df.label=='other topics']
df_imbalanced3 = df[df.label=='purchase']
df_imbalanced4 = df[df.label=='recruit']

In [18]:
df_unsampled1 = resample(df_imbalanced1, replace=True, n_samples=2552, random_state=42)
df_unsampled2 = resample(df_imbalanced2, replace=True, n_samples=2552, random_state=42)
df_unsampled3 = resample(df_imbalanced3, replace=True, n_samples=2552, random_state=42)
df_unsampled4 = resample(df_imbalanced4, replace=True, n_samples=2552, random_state=42)

In [19]:
df_unsampled = pd.concat([df_majority, df_unsampled1])
df_unsampled = pd.concat([df_unsampled, df_unsampled2])
df_unsampled = pd.concat([df_unsampled, df_unsampled3])
df_unsampled = pd.concat([df_unsampled, df_unsampled4])

In [20]:
df_unsampled.label.value_counts()

advertisement    2552
other topics     2552
purchase         2552
sentiment        2552
recruit          2552
Name: label, dtype: int64

In [21]:
def split_data(data, test_ratio):
    shuffled_id = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_ids = shuffled_id[:test_set_size]
    train_ids = shuffled_id[test_set_size:]
    return data.iloc[train_ids], data.iloc[test_ids]

# Model

In [22]:
train_set, test_set = split_data(df_unsampled, 0.2)

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

In [24]:
clf = Pipeline([
    ('cv', CountVectorizer(stop_words=None, tokenizer=None,
                          preprocessor=None)),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                         alpha=0.01, random_state=42,
                         max_iter=5, tol=None))
])

In [25]:
clf.fit(train_set.content, train_set.label)

/home/anhduc/environments/my_env/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('cv', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_a...dom_state=42, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))])

In [26]:
pred = clf.predict(test_set.content)

In [27]:
np.mean(pred==test_set.label)

0.8726489028213166

In [28]:
from sklearn.model_selection import cross_val_score

In [29]:
scores = cross_val_score(clf, train_set.content, train_set.label, cv=5)
scores.mean()

/home/anhduc/environments/my_env/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


0.876273418726171

In [30]:
from sklearn.metrics import f1_score

In [31]:
f1_score(test_set.label, pred, average='weighted')

0.8692983539859607